In [28]:
import pandas as pd
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

# Experimental data
k_1 = 1
k_2 = 4
t_exp = np.array([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])
u_0_exp = np.array([np.exp(-k_1*t) for t in t_exp])
u_1_exp = np.array([k_1/(k_2-k_1)*(np.exp(-k_1*t)-np.exp(-k_2*t)) for t in t_exp])
u_exp = np.column_stack((u_0_exp, u_1_exp))
obs_data = pd.DataFrame({'t': t_exp, 'u_0': u_0_exp, 'u_1': u_1_exp})

del k_1, k_2

# Build ANN
import tensorflow as tf
from tensorflow import keras

# Define the model architecture
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08) 
model = keras.Sequential([
    keras.layers.Dense(5, activation='tanh', input_shape=(1,)),
    keras.layers.Dense(2)
])

# Compile the model
model.compile(optimizer=optimizer, loss='mae')
history = model.fit(t_exp, np.column_stack((u_0_exp, u_1_exp)), epochs=10000, verbose=0)

#### Start from here

In [39]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.integrate import odeint

try:
    obs_data = pd.read_csv('ANN_data_old.csv', index_col=False)
except FileNotFoundError:    
    obs_data['u_0_pred'] = model.predict(t_exp)[:,0]
    obs_data['u_1_pred'] = model.predict(t_exp)[:,1]
    obs_data['du0_dt'] = np.gradient(obs_data['u_0_pred'], obs_data['t'])
    obs_data['du1_dt'] = np.gradient(obs_data['u_1_pred'], obs_data['t'])
    obs_data.to_csv('ANN_data.csv', index=False)

obs_data

,t,u_0,u_1,u_0_pred,u_1_pred,du0_dt,du1_dt
0,0.0,1.000000,-0.000000,0.999002,0.000486,-3.796915,3.558243
1,0.1,0.606531,0.372883,0.619310,0.356311,-3.157286,2.815378
2,0.2,0.367879,0.563564,0.367545,0.563562,-2.006304,1.486933
3,0.3,0.223130,0.647110,0.218050,0.653697,-1.176545,0.547188
4,0.4,0.135335,0.668731,0.132236,0.673000,-0.680613,0.008904
5,0.5,0.082085,0.655557,0.081927,0.655478,-0.406196,-0.260511
6,0.6,0.049787,0.623781,0.050996,0.620898,-0.253865,-0.379319
7,0.7,0.030197,0.582985,0.031154,0.579614,-0.163413,-0.420609
8,0.8,0.018316,0.538767,0.018314,0.536776,-0.103556,-0.423863
9,0.9,0.011109,0.494326,0.010443,0.494842,-0.059301,-0.409285


In [41]:
np.gradient(obs_data['u_0_pred'], obs_data['t'])

array([-3.7969154 , -3.1572862 , -2.00630345, -1.17654465, -0.680613  ,
       -0.40619604, -0.25386482, -0.16341269, -0.10355562, -0.05930126,
       -0.0398925 ])

In [40]:
np.array([-5 * obs_data['u_0_pred'].values, 5 * obs_data['u_0_pred'].values - 1 * obs_data['u_1_pred'].values]).T


array([[-4.995009  ,  4.99452264],
       [-3.0965513 ,  2.74024066],
       [-1.8377228 ,  1.27416076],
       [-1.09024785,  0.43655065],
       [-0.66117815, -0.01182159],
       [-0.40963485, -0.24584309],
       [-0.25498212, -0.36591542],
       [-0.15577004, -0.42384416],
       [-0.09156942, -0.44520627],
       [-0.05221442, -0.44262725],
       [-0.03226817, -0.4226506 ]])

In [25]:
# Define function to minimize
def L(x):
    k_1, k_2 = x
    
    duANN_dt = np.column_stack((obs_data['du0_dt'], obs_data['du1_dt']))
    f_ANN =  np.array([-k_1 * obs_data['u_0_pred'].values, k_1 * obs_data['u_0_pred'].values - k_2 * obs_data['u_1_pred'].values]).T
    return 0.5 * np.sum((duANN_dt - f_ANN)**2)

# Initial guess
x0 = [1, 1]

# Minimize
res = minimize(L, x0, method='Nelder-Mead')
res.x

array([4.19147144, 0.72436078])

In [26]:
def P_1(p):
    k_1, k_2 = p
    def model_ode(z, t):
        u_0, u_1 = z
        return [-k_1*u_0, k_1*u_0 - k_2*u_1]
    
    exp = np.column_stack((obs_data['u_0'], obs_data['u_1']))
    z_ode = odeint(model_ode,  [1,0], obs_data['t'])
    return 0.5 * np.sum((exp - z_ode)**2)

x0 = [res.x[0], res.x[1]]
# x0 = [1,1]
res = minimize(P_1, x0, method='Nelder-Mead')
res.x
    

array([5.00003592, 1.00000737])